In [2]:
# Python routine to read position of motion controller and resulting voltage from voltmeter

#Setup block. Run this once to import the libraries and files we will need.
%pylab qt
import keyboard
import matplotlib.pyplot as plt #For plotting
from matplotlib.animation import FuncAnimation # For real time display
import winsound #For making the computer beep
from time import time, sleep, localtime, strftime  #Timekeeping functions
import csv #For writing the data to a file (easier than writing with pandas when appending to file)
import pandas as pd #For reading the data from a file (easier than reading with csv)

import pyvisa as visa #For communicating with instruments
import pyvisa.constants  #needed to access comm buffers


Populating the interactive namespace from numpy and matplotlib


In [3]:
#Set up resources

rm = visa.ResourceManager()
print("Here are the resources in use: ")
print(rm.list_resources())
resources = rm.list_resources()

voltmeterName = "5491B  Multimeter"
mocontName = "MM3000"
fields = ['Position', 'Voltage']


mocont = rm.open_resource('ASRL1::INSTR')
mocont.timeout = 4000
print("\nOpening resource ", mocont)
mocont.read_termination = '\r\n'
mocont.write_termination = '\r'
mocont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
mocont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
sleep(2)
    
try:
    return_str = mocont.query('VE') # checks the name of the device

    sleep(0.5)
    print('VE response: ',return_str)
    #Check if usable resources are multimeters
    #   if they are, assign the ammeter first
    if mocontName in return_str:
        print("\nMotion Controller found: ", return_str)
        mocont.timeout = 3000
        mocont.close()

except Exception as E:
    errorname = str(E).rsplit(' ')[0]
    print(" unavailable; ", E)
    mocont.close()


resList=['ASRL1::INSTR']
voltmeter = None

#Look for appropriate resources
print("Opening them in turn and asking who they are: ")
for i in resources:
    if (i in resList):  # Don't look at instrument already found
        continue
    print("Resource: ", i, end='')
    
    #Check if resources are usable
    
    res_item = rm.open_resource(i)
    res_item.timeout = 1000
    res_item.baud_rate = 9600
    print("\nOpening resource: ", res_item)
    res_item.read_termination = '\n'
    res_item.write_termination = '\n'
    res_item.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
    res_item.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
    sleep(2)
    print("Asking for identification...")
    
    try:
        return_str = res_item.query('*IDN?') # checks the name of the device

            
        print('*IDN? response: ',return_str)
        #Check if usable resources are multimeters
        #   if they are, assign the ammeter first
        if voltmeterName in return_str:
            voltmeter = res_item
            resList.append(voltmeter)
            print("\nAmmeter found: ", return_str)
            res_item.close()
            continue
            
    
    except Exception as E:
        errorname = str(E).rsplit(' ')[0]
        print(" unavailable; ", E)
        res_item.close()
        continue

Here are the resources in use: 
('ASRL1::INSTR', 'ASRL4::INSTR')

Opening resource  SerialInstrument at ASRL1::INSTR
VE response:  Newport Corp. MM3000 Version 3.0 

Motion Controller found:  Newport Corp. MM3000 Version 3.0 
Opening them in turn and asking who they are: 
Resource:  ASRL4::INSTR
Opening resource:  SerialInstrument at ASRL4::INSTR
Asking for identification...
*IDN? response:  5491B  Multimeter,Ver1.4.14.06.18,124E16149

Ammeter found:  5491B  Multimeter,Ver1.4.14.06.18,124E16149


In [77]:
delay = 1
startingPos = -500
endingPos = 1000
stepSize = 1

voltmeter.open()
mocont.open()
sleep(1)
mocont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
mocont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
voltmeter.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)

now = strftime("%d%b%y_%I-%M-%S",localtime()) #grab the current time and put it in the filename
filename = "FresnelDiffraction_" + now + "A.csv"
print("Data will be written to " + filename)

f_out = open(filename, 'w', newline='')
datafile = csv.writer(f_out, delimiter=',')
datafile.writerow(fields) #Write header to csv

def stepsToMM(steps):
    return (float(steps)*0.0555) # Convert steps to μm (Assuming step amount is 0.0000555 mm)

def initMoCont():
    try:
        mocont.write('1US0.055um')
        mocont.write('1UUmm')
        mocont.write('1UV0.3')
        mocont.write('1UA1.2')
        if(startingPos < 0):
            mocont.write(f'1UP-{abs(startingPos/1000)}')
        else:
            mocont.write(f'1UP+{abs(startingPos/1000)}')
        mocont.write('1FE+500')
        mocont.write('1IL+100')
        mocont.write('1KP+1500')
        mocont.write('1KD+5000')
        mocont.write('1KI+100')
        
    except:
        mocont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
        mocont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
        print("Error")
    print("Motion Contoller Initialized")
    
def IteratePos():
    try:
        mocont.write(f'1UR+{stepSize/1000}')
    except:
        mocont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
        mocont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
        print("Error")
        
def formatTime(t):
    if(t < 1):
        return "0secs"
    output = ""
    if(int(int(t)/3600) > 0):
        output += f"{int(int(t)/3600):04d}hrs "
    else:
        output += ""
    if(int((int(t) % 3600) / 60) > 0):
        output += f"{int((int(t) % 3600) / 60):04d}mins "
    else:
        output += ""
    output += f"{(t - int((int(t)/3600))*3600 - int((int(t) % 3600) / 60)*60):02.2f}secs "
    return output

initMoCont()
sleep(5)
    
#initialize the animated plot
fig, ax = plt.subplots()
plt.xlabel("Position (μm)")
plt.ylabel("Voltage (V)")
xdata, ydata = [], []
ln, = plot([], [], '-')


print("Starting the scan")
print("Live plot may appear in the task bar at bottom of screen")
sleep(0.25)
startTime = time()
mocont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
mocont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
#live update of plot here
count = 0
done_flag = False
def update(count, done_flag):
    while(done_flag == False): 
        
        voltage = float(voltmeter.query(':FETC?'))
        position = stepsToMM(int(str(mocont.query('1TPI')).rsplit(' ')[0]))
        IteratePos()
      
        xdata.append(position)
        ydata.append(voltage)
        datafile.writerow([position, voltage])
        
        elapsedTime = time() - startTime
        print(f"Current Position: {position:07.2f}μm Elapsed Time: {formatTime(elapsedTime)} Est Remaining Time: {formatTime(elapsedTime/((position-startingPos)/(endingPos-startingPos))-elapsedTime)}                       ", end="\r", flush=True)
        


        ln.set_data(xdata, ydata)
        ax.relim() 
        ax.autoscale_view()
        plt.xlim([startingPos, endingPos])
        title("Voltage vs. Position")
        count += 1
        sleep(0.2)  #dwell time at each data point.  Below 0.2 doesn't leave time to plot
        
        if position > endingPos: 
            print(flush = False)
            f_out.close()
            print("Finished gathering data")
            done_flag = True
            sleep(0.2)
            ani.event_source.stop()
            plt.close()
            figure()
            title("Voltage vs. Position")
            plt.xlabel("Position (μm)")
            plt.ylabel("Voltage (V)")
            plot(xdata,ydata,'-')
            
            
            
        else:
            return ln,
ani = FuncAnimation(fig, update,interval = 500,fargs = [done_flag])        
#ani = FuncAnimation(fig, update)

plt.show()
        

Data will be written to FresnelDiffraction_21Jun22_11-20-55A.csv
Motion Contoller Initialized
Starting the scan
Live plot may appear in the task bar at bottom of screen
Current Position: 1000.94μm Elapsed Time: 0021mins 18.94secs  Est Remaining Time: 0secs                                     
Finished gathering data
